## Imports and reading in data

In [2]:
#imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sns
%matplotlib inline

In [146]:
#Paths to files
data_path = 'Q:/hackers09/shared/data/'
cancer_path = os.path.join(data_path, 'df_cancer.csv')
echo_path = os.path.join(data_path, 'df_echo.csv')
encounter_path = os.path.join(data_path, 'df_encounter.csv')
labs_path = os.path.join(data_path, 'df_labs.csv')
outcome_path = os.path.join(data_path, 'df_outcome.csv')
problist_path = os.path.join(data_path, 'df_problist.csv')
radiology_path = os.path.join(data_path, 'df_radiology.csv')
registry_path = os.path.join(data_path, 'df_registry.csv')
vitals_path = os.path.join(data_path, 'df_vitals.csv')

In [156]:
#Read in dataframes
cancer_df = pd.read_csv(cancer_path, encoding='ISO-8859-1')
cancer_df.set_index("HSP_ENC", inplace = True)
echo_df = pd.read_csv(echo_path, encoding='ISO-8859-1')
echo_df.set_index("HSP_ENC", inplace = True)
encounter_df = pd.read_csv(encounter_path, encoding='ISO-8859-1')
encounter_df.set_index("HSP_ENC", inplace = True)
labs_df = pd.read_csv(labs_path, encoding='ISO-8859-1')
labs_df.set_index("HSP_ENC", inplace = True)
outcome_df = pd.read_csv(outcome_path, encoding='ISO-8859-1')
outcome_df.set_index("HSP_ENC", inplace = True)
problist_df = pd.read_csv(problist_path, encoding='ISO-8859-1')
problist_df.set_index("HSP_ENC", inplace = True)
radiology_df = pd.read_csv(radiology_path, encoding='ISO-8859-1')
radiology_df.set_index("HSP_ENC", inplace = True)
registry_df = pd.read_csv(registry_path, encoding='ISO-8859-1')
registry_df.set_index("HSP_ENC", inplace = True)
vitals_df = pd.read_csv(vitals_path, encoding='ISO-8859-1')
vitals_df.set_index("HSP_ENC", inplace = True)

##  
## Define functions to merge datasets

In [152]:
#function to merge cancer data
def merge_cancer(enc_df, cnc_df):
    #Make modifications to cancer df
    mod_cancer_df = cnc_df.copy()
#     mod_cancer_df.set_index("HSP_ENC", inplace = True)
    mod_cancer_df.reset_index(level=0, inplace=True)
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Missing Remission Date', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Previously Positive', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not Documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'No Cancer', 'cancer_at_enc'] = '1'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown', 'cancer_at_enc'] = '2'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Cancer', 'cancer_at_enc'] = '3'
    mod_cancer_df['cancer_at_enc'] = mod_cancer_df['cancer_at_enc'].astype(int)
    mod_cancer_df.drop_duplicates(['PATIENT_ID', 'HSP_ENC'])
    
    #Take only cancer status column with max value
    drop_cancer_df = mod_cancer_df[['HSP_ENC', 'cancer_at_enc']]
    drop_cancer_df = drop_cancer_df.groupby('HSP_ENC',group_keys=False).apply(lambda x: x.loc[x['cancer_at_enc']==x['cancer_at_enc'].max()])

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, drop_cancer_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')

    #Replace NaN in encounter df with 0 (Never had cancer)
    mergeRes['cancer_at_enc'].fillna(0, inplace=True)
    
    #Renaming the cancer column
    mergeRes.rename(columns={'cancer_at_enc': 'CANCER_RANK'}, inplace=True)
    
    mergeRes.reset_index(drop=True)
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

In [154]:
def merge_registry(enc_df, reg_df):
    reg_list = reg_df.REGISTRY_NAME.unique().tolist()
    reg_df_copy = reg_df.copy()
    reg_df_copy.reset_index(level=0, inplace=True)

    tst_df = reg_df_copy[['HSP_ENC']].copy()
    for item in reg_list:
        tst_df[item] = False
    tst_df = tst_df.drop_duplicates()


    for index, row in reg_df_copy.iterrows():
        enc_id = reg_df_copy.iloc[index, 0]
        curr_reg = reg_df_copy.iloc[index, 2]
        tst_df.loc[tst_df['HSP_ENC'] == enc_id, [curr_reg]] = True

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, tst_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')
    
    #Replace NaN in encounter df with False, no record
    for item in reg_list:
        mergeRes[item].fillna(False, inplace=True)
    
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

##  
## Start calling merge functions

In [158]:
encounter_df = merge_registry(encounter_df, registry_df)

,PATIENT_ID,ED_EPISODE_ID,ED_DISP,DISCH_DISP,ADT_ARRIVAL_TIME_DIFFSEC,ED_DISP_TIME_DIFFSEC,HOSP_DISCH_TIME_DIFFSEC,ADMIT_SOURCE,ADT_PAT_CLASS,HOSP_SERVICE,...,GENERAL MALIGNANCY REGISTRY,CHRONIC LUNG REGISTRY,ASTHMA REGISTRY,CORONARY ARTERY DISEASE REGISTRY,CHRONIC HEART REGISTRY,SLEEP APNEA REGISTRY,LUNG CANCER REGISTRY,LIVER CIRRHOSIS REGISTRY,CRANIOTOMY REGISTRY,CHRONIC RENAL FAILURE REGISTRY
HSP_ENC,,,,,,,,,,,,,,,,,,,,,
260755660,1305,34382396,Admitted,To Home Or Self Care,-420,10500,627600,Home & Outside Location,Inpatient,Pulmonary,...,True,True,True,False,False,False,False,False,False,False
192470437,1261,27636517,Admitted,To Home With Home Health,-120,9120,516300,Home & Outside Location,Inpatient,Cardiology,...,False,False,False,True,False,False,False,False,False,False
258754156,785,34189980,Admitted,Expired,0,40200,899640,Home & Outside Location,Inpatient,Cardiology,...,False,True,False,True,True,False,False,False,False,False
306050512,504,38409274,Admitted,To Home Or Self Care,0,7860,388440,Home & Outside Location,Inpatient,Internal Medicine,...,False,True,True,False,False,False,False,False,False,False
163297609,631,22833060,Admitted,To Rehab Unit Zale Lipshy,0,18300,758880,Home & Outside Location,Inpatient,Hematology-Oncology,...,True,True,True,False,True,True,False,False,False,False
289108153,573,37061739,Admitted,To Hospice Inpatient Care,0,17520,2819040,Home & Outside Location,Inpatient,Internal Medicine,...,False,False,False,True,False,False,False,False,False,False
241757680,972,32759105,Admitted,To Home Or Self Care,0,14520,164160,Home & Outside Location,Inpatient,Internal Medicine,...,False,True,False,False,False,False,False,False,False,False
277292452,1299,35925592,Admitted,To Home Or Self Care,-360,16200,369300,Home & Outside Location,Inpatient,Internal Medicine,...,False,True,True,False,False,False,False,False,False,False
1831174945,282,41245461,Admitted,To Home Or Self Care,-240,25500,600600,Home & Outside Location,Inpatient,Internal Medicine,...,False,False,False,True,False,False,False,False,False,False
